### in this script we prepare the data for the glm model, were we calculate distance per stage per pair, between all session combinations for whole, only partner and only non partner data

we first process the sequnce data

In [ ]:

# %% ---------------------------------------------------------------------------
# 0)  Imports & configuration  
# ---------------------------------------------------------------------------
import os, itertools, numpy as np, pandas as pd
from scipy import stats
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from itertools import combinations
from scipy.spatial.distance import squareform
from scipy.stats import pearsonr, spearmanr
from scipy.spatial.distance import pdist, squareform     
from scipy.spatial import distance as ssd

BASE_DIR      = Path.cwd().parent
SEQ_DATA_DIR  = BASE_DIR / "seqeunce data"

CSV_PATH      = SEQ_DATA_DIR / "Processed_seq_data.csv"     # master CSV

PAIRS = {"Tabor":"Lola","Odin":"Nougatti","Wuschel":"Olympia"} | \
        {"Lola":"Tabor","Nougatti":"Odin","Olympia":"Wuschel"}

MIN_SEQ_PER_LIST = 5
NGRAM_N, ELEMENT_OF_INT, MAX_REPEAT_LEN = 2, "A", 5
OUT_BASENAME = "glm_data_seq_{}.csv"            # csv of all distances

OUTPUT_DIR    = BASE_DIR / "glm data"                       # ✦ NEW
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)



# %% ---------------------------------------------------------------------------
# 1)  Metric functions  (identical to originals)
# ------------------------------------------------------------------------------

def smith_waterman(sequence1, sequence2, match_score=1, mismatch_score=-2, gap_penalty=-2):
    m, n = len(sequence1), len(sequence2)
    score_matrix = [[0]*(n+1) for _ in range(m+1)]
    traceback_matrix = [['']*(n+1) for _ in range(m+1)]
    max_score, max_pos = 0, (0, 0)

    for i in range(1, m+1):
        for j in range(1, n+1):
            match  = score_matrix[i-1][j-1] + (match_score if sequence1[i-1]==sequence2[j-1] else mismatch_score)
            delete = score_matrix[i-1][j]   + gap_penalty
            insert = score_matrix[i][j-1]   + gap_penalty
            score_matrix[i][j] = max(0, match, delete, insert)

            if score_matrix[i][j] == match:   traceback_matrix[i][j] = '\\'
            elif score_matrix[i][j] == delete: traceback_matrix[i][j] = '^'
            elif score_matrix[i][j] == insert: traceback_matrix[i][j] = '<'

            if score_matrix[i][j] > max_score:
                max_score, max_pos = score_matrix[i][j], (i, j)

    return max_score


def distance_local_alignment_internal(seqs1, seqs2):
    scores = []
    for s1 in seqs1:
        best = float('-inf')
        for s2 in seqs2:
            sc = smith_waterman(s1, s2) / np.log(len(s2))
            best = max(best, sc)
        scores.append(best)
    return -np.sqrt(np.mean(np.square(scores)))


def distance_local_alignment(seqs1, seqs2):
    return (distance_local_alignment_internal(seqs1, seqs2) +
            distance_local_alignment_internal(seqs2, seqs1)) / 2


def calculate_transition_matrix_(sequences, alphabet):
    tm = pd.DataFrame(0, index=list(alphabet), columns=list(alphabet))
    for seq in sequences:
        for i in range(len(seq)-1):
            tm.at[seq[i], seq[i+1]] += 1
    return tm.div(tm.sum(axis=1), axis=0)


def distance_transition_matrix(seqs1, seqs2, alphabet):
    m1 = calculate_transition_matrix_(seqs1, alphabet).to_numpy().flatten()
    m2 = calculate_transition_matrix_(seqs2, alphabet).to_numpy().flatten()
    return np.linalg.norm(np.nan_to_num(m1) - np.nan_to_num(m2))


def calculate_ngram_prob_distribution(sequences, n, all_unique_ngrams):
    counts = {ng: 0 for ng in all_unique_ngrams}
    for seq in sequences:
        ngrams = [tuple(seq[i:i+n]) for i in range(len(seq)-n+1)]
        for ng in ngrams: counts[ng] += 1
    tot = sum(counts.values()) + 1e-3
    return {ng: c/tot for ng, c in counts.items()}


def distance_n_gram(seqs1, seqs2, n, alphabet):
    all_ngrams = [tuple(''.join(t)) for t in itertools.product(alphabet, repeat=n)]
    v1 = np.array([calculate_ngram_prob_distribution([s for s in seqs1 if len(s)>=n], n, all_ngrams)[ng] for ng in all_ngrams])
    v2 = np.array([calculate_ngram_prob_distribution([s for s in seqs2 if len(s)>=n], n, all_ngrams)[ng] for ng in all_ngrams])
    return np.linalg.norm(v1 - v2)


def calculate_repeat_length_prob_distribution_specific(sequences, max_len, element):
    counts = {l: 0 for l in range(2, max_len+1)}
    for seq in sequences:
        i = 0
        while i < len(seq):
            cnt = 1
            while i+1 < len(seq) and seq[i]==seq[i+1]:
                i += 1; cnt += 1
            if 1 < cnt <= max_len and seq[i] in element: counts[cnt] += 1
            i += 1
    tot = sum(counts.values()) + 1e-3
    return {l: c/tot for l, c in counts.items()}


def distance_repeat_specific(seqs1, seqs2, element, max_len=5):
    p1 = calculate_repeat_length_prob_distribution_specific(seqs1, max_len, element)
    p2 = calculate_repeat_length_prob_distribution_specific(seqs2, max_len, element)
    lens = sorted(set(p1)|set(p2))
    v1 = np.array([p1.get(l,0) for l in lens])
    v2 = np.array([p2.get(l,0) for l in lens])
    return np.linalg.norm(v1 - v2)


# %% ---------------------------------------------------------------------------
# 2)  Helper: build one distance table  (only change: session_number)
# ---------------------------------------------------------------------------
def build_distance_table(df_subset: pd.DataFrame) -> pd.DataFrame:
    grouped = (
        df_subset
        .groupby(['focal ID', 'stage', 'session_number'])['true_sequence']
        .apply(list)
        .to_dict()
    )
    grouped = {k:v for k,v in grouped.items() if len(v) >= MIN_SEQ_PER_LIST}
    alphabet = set(''.join(df_subset['true_sequence']))
    rows = []

    for ind1, ind2 in [('Tabor','Lola'), ('Odin','Nougatti'), ('Wuschel','Olympia')]:
        for stage in ['before','after']:
            sess1 = [s for (_,st,s) in grouped if _==ind1 and st==stage]
            sess2 = [s for (_,st,s) in grouped if _==ind2 and st==stage]
            for s1 in sess1:
                seq1 = grouped[(ind1,stage,s1)]
                for s2 in sess2:
                    seq2 = grouped[(ind2,stage,s2)]
                    rows.extend([
                        dict(pair=f"{ind1}-{ind2}", stage=stage,
                             session_focal=s1, session_partner=s2,
                             metric='local_alignment',
                             distance=distance_local_alignment(seq1,seq2)),
                        dict(pair=f"{ind1}-{ind2}", stage=stage,
                             session_focal=s1, session_partner=s2,
                             metric='transition_matrix',
                             distance=distance_transition_matrix(seq1,seq2,alphabet)),
                        dict(pair=f"{ind1}-{ind2}", stage=stage,
                             session_focal=s1, session_partner=s2,
                             metric='bigram',
                             distance=distance_n_gram(seq1,seq2,NGRAM_N,alphabet)),
                        dict(pair=f"{ind1}-{ind2}", stage=stage,
                             session_focal=s1, session_partner=s2,
                             metric='repeat_A_len',
                             distance=distance_repeat_specific(seq1,seq2,ELEMENT_OF_INT,MAX_REPEAT_LEN)),
                    ])
    df = pd.DataFrame(rows)
    # ── normalisations ──────────────────────────────────────────────
    df['z_distance'] = (
        df.groupby('metric')['distance']
        .transform(lambda x: (x - x.mean()) / x.std(ddof=0)
                            if x.std(ddof=0) else 0.0)
    )

    df['distance_01'] = (
        df.groupby('metric')['distance']
        .transform(lambda x: (x - x.min()) / (x.max() - x.min())
                            if (x.max() - x.min()) else 0.0)
    )  # min–max scaling 0‥1


    return df


# %% ---------------------------------------------------------------------------
# 4)  Run pipeline for each subset, save distance & summary tables
# ---------------------------------------------------------------------------

seq_data = pd.read_csv(CSV_PATH)          # typo fixed (was FILE_PATH)

subsets = {
    "whole":      seq_data,
    "partner":     seq_data[seq_data['paired_status']=="partner"],
    "non_partner": seq_data[seq_data['paired_status']=="non-partner"],
}

for label, df_sub in subsets.items():
    print(f"\n=== {label.upper()} ({len(df_sub)} rows) ===")
    dist_tbl = build_distance_table(df_sub)

    outfile = OUTPUT_DIR / OUT_BASENAME.format(label)       # 
    dist_tbl.to_csv(outfile, index=False)                   # 
    print(f"saved {outfile}  |  {len(dist_tbl)} comparisons")




=== WHOLE (1619 rows) ===
saved c:\Users\nakul\Desktop\marmoset project\2025_Work_Re-start_Marmoset\2025\glm data\glm_data_seq_whole.csv  |  628 comparisons

=== PARTNER (476 rows) ===
saved c:\Users\nakul\Desktop\marmoset project\2025_Work_Re-start_Marmoset\2025\glm data\glm_data_seq_partner.csv  |  248 comparisons

=== NON_PARTNER (1143 rows) ===
saved c:\Users\nakul\Desktop\marmoset project\2025_Work_Re-start_Marmoset\2025\glm data\glm_data_seq_non_partner.csv  |  500 comparisons


we now process the spectral data

In [7]:
# %% ---------------------------------------------------------------------------
# Spectral Convergence Pipeline – run in VS Code or Jupyter
# ---------------------------------------------------------------------------
# Mirrors the syntactic‑distance workflow but uses spectral features:
#   • Euclidean distance on 5 PCs of traditional acoustics
#   • Euclidean distance on 5 PCs of MFCCs (columns 1‑132)
#   • DTW alignment‑cost of 4–12 kHz spectrograms
# ---------------------------------------------------------------------------

# %% ---------------------------------------------------------------------------
# Imports & basic configuration
# ---------------------------------------------------------------------------
import os, itertools, random, pathlib
from functools import lru_cache

import numpy as np
import pandas as pd
from scipy import stats, spatial
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from tqdm import tqdm                      # <‑‑ progress bars

import librosa
from pydub import AudioSegment
from fastdtw import fastdtw
from scipy.spatial.distance import cosine

# %% ---------------------------------------------------------------------------
# Paths & constants  —— tweak here if directories move
# ---------------------------------------------------------------------------

BASE_DIR      = Path.cwd().parent
SPEC_DATA_DIR  = BASE_DIR / "spectral data"

CSV_PATH      = SPEC_DATA_DIR / "Processed_spec_data.csv" 
df = pd.read_csv(CSV_PATH )

# pick a sensible base directory whether running as notebook or script
                        # running in Jupyter

OUTPUT_DIR   = BASE_DIR / "glm data"
OUTPUT_DIR.mkdir(exist_ok=True)

OUT_BASENAME = OUTPUT_DIR / "glm_data_spec_{}.csv"

# partner mapping (bidirectional)
PAIRS = {"Tabor":"Lola", "Odin":"Nougatti", "Wuschel":"Olympia"}
PAIRS |= {v: k for k, v in PAIRS.items()}

# analysis parameters
MIN_CALLS_PER_LIST = 5
DTW_BAND_LO        = 4_000     # Hz
DTW_BAND_HI        = 12_000    # Hz
SHOW_PROGRESS      = True      # toggle tqdm progress bars


# %% ---------------------------------------------------------------------------
# 2)  Distance‑metric helpers
# ---------------------------------------------------------------------------

# put this near the top, after imports
def _count_session_pairs(grouped, ind1, ind2, stage):
    sess1 = [k[2] for k in grouped if k[0] == ind1 and k[1] == stage]
    sess2 = [k[2] for k in grouped if k[0] == ind2 and k[1] == stage]
    return len(sess1) * len(sess2)

def dist_pc_pairwise(list1: pd.DataFrame, list2: pd.DataFrame, pc_cols):
    """Mean pairwise Euclidean distance between two call lists on the given PCs."""
    dists = spatial.distance.cdist(list1[pc_cols], list2[pc_cols], metric="euclidean")
    return float(dists.mean())


# -------  DTW helpers (spectrogram → alignment cost)  ------------------------


def generate_spectrogram(audio_file_path, pad_duration=0):
    """Return dB‑scaled spectrogram (freq × time) and its sample rate."""
    audio = AudioSegment.from_wav(str(audio_file_path))
    y = np.array(audio.get_array_of_samples(), dtype=np.float32)
    sr = audio.frame_rate
    if audio.channels == 2:
        y = y.reshape((-1, 2)).mean(axis=1)              # stereo → mono

    # symmetric zero‑pad if shorter than target
    target_len = int(pad_duration * sr)
    if len(y) < target_len:
        pad = target_len - len(y)
        y = np.pad(y, (pad // 2, pad - pad // 2), mode="constant")

    N_FFT      = 2048
    HOP_LENGTH = N_FFT // 4
    WIN_LENGTH = N_FFT
    D = librosa.stft(y, n_fft=N_FFT, hop_length=HOP_LENGTH, win_length=WIN_LENGTH, window="hann")
    S_db = librosa.amplitude_to_db(np.abs(D), ref=np.max)
    return S_db, sr


def crop_band(S_db, sr, lo=DTW_BAND_LO, hi=DTW_BAND_HI, n_fft=2048):
    """Keep only rows within [lo, hi] Hz."""
    freqs = librosa.fft_frequencies(sr=sr, n_fft=n_fft)
    band   = (freqs >= lo) & (freqs <= hi)
    return S_db[band, :]


def compute_alignment_cost(S1, S2):
    """DTW alignment cost (cosine distance) normalised by total frames."""
    dist, _ = fastdtw(S1.T, S2.T, dist=cosine)
    return dist / (S1.shape[1] + S2.shape[1])


# cache spectrograms so we don’t re‑load the same file a hundred times
@lru_cache(maxsize=2048)
def _spectro_cache(path: str):
    S_db, sr = generate_spectrogram(path, pad_duration=0.5)
    return crop_band(S_db, sr)


def dist_dtw_pairwise(list1: pd.DataFrame, list2: pd.DataFrame):
    """Mean DTW alignment‑cost between every call in list1 × list2, with a progress bar."""
    costs = []
    n1, n2 = len(list1), len(list2)
    total  = n1 * n2

    iterator1 = list1["call_audio_path"].tolist()
    iterator2 = list2["call_audio_path"].tolist()

    use_bar = SHOW_PROGRESS and total > 1
    pbar = tqdm(total=total, desc="DTW pairwise", unit="align", leave=False) if use_bar else None

    try:
        for p1 in iterator1:
            S1 = _spectro_cache(p1)
            for p2 in iterator2:
                S2 = _spectro_cache(p2)
                costs.append(compute_alignment_cost(S1, S2))
                if pbar is not None:
                    pbar.update(1)
    finally:
        if pbar is not None:
            pbar.close()

    return float(np.mean(costs))


# %% ---------------------------------------------------------------------------
# 4)  Build distance table for a given subset (whole / paired / non‑paired)
# ---------------------------------------------------------------------------
from contextlib import nullcontext     # add with other imports

def build_distance_table_spectral(df_subset: pd.DataFrame) -> pd.DataFrame:
    # --- group calls by (focal, stage, session) -----------------------------
    grouped = {
        k: g for k, g in df_subset.groupby(
            ["focal ID", "stage", "session_number"]
        ) if len(g) >= MIN_CALLS_PER_LIST
    }

    # ---------- NEW: pre-compute how many session pairs will need DTW -------
    total_pairs = sum(
        _count_session_pairs(grouped, ind1, ind2, stage)
        for ind1, ind2 in [("Tabor", "Lola"),
                           ("Odin",  "Nougatti"),
                           ("Wuschel", "Olympia")]
        for stage in ["before", "after"]
    )
    if SHOW_PROGRESS:
        print(f"  → DTW will be computed for {total_pairs} session pairs")

    outer_bar = tqdm(total=total_pairs, desc="DTW session-pairs",
                     unit="pair", leave=False) if SHOW_PROGRESS else nullcontext()

    rows = []
    try:
        for ind1, ind2 in [("Tabor", "Lola"),
                           ("Odin",  "Nougatti"),
                           ("Wuschel", "Olympia")]:
            for stage in ["before", "after"]:
                sess1 = [k[2] for k in grouped if k[0] == ind1 and k[1] == stage]
                sess2 = [k[2] for k in grouped if k[0] == ind2 and k[1] == stage]
                for s1 in sess1:
                    L1 = grouped[(ind1, stage, s1)]
                    for s2 in sess2:
                        L2 = grouped[(ind2, stage, s2)]
                        sess_pair = f"({s1},{s2})"

                        # ── Euclidean distances on PCs ─────────────────────
                        rows += [
                            dict(pair=f"{ind1}-{ind2}", stage=stage,
                                 session_focal=s1, session_partner=s2,
                                 sess_pair=sess_pair,
                                 metric="T_PC_pairwise",
                                 distance=dist_pc_pairwise(L1, L2,
                                                           [f"trad_PC{i}" for i in range(1, 6)])),
                            dict(pair=f"{ind1}-{ind2}", stage=stage,
                                 session_focal=s1, session_partner=s2,
                                 sess_pair=sess_pair,
                                 metric="M_PC_pairwise",
                                 distance=dist_pc_pairwise(L1, L2,
                                                           [f"mfcc_PC{i}" for i in range(1, 6)])),
                        ]

                        # ── DTW – slow part, keep last to benefit from bar ─
                        rows.append(
                            dict(pair=f"{ind1}-{ind2}", stage=stage,
                                 session_focal=s1, session_partner=s2,
                                 sess_pair=sess_pair,
                                 metric="spectro_DTW",
                                 distance=dist_dtw_pairwise(L1, L2))
                        )

                        if SHOW_PROGRESS:
                            outer_bar.update(1)
    finally:
        if SHOW_PROGRESS:
            outer_bar.close()

    # ---------- normalise ---------------------------------------------------
    dist_df = pd.DataFrame(rows)
    dist_df["z_distance"] = dist_df.groupby("metric")["distance"] \
        .transform(lambda x: (x - x.mean()) / x.std(ddof=0) if x.std(ddof=0) else 0.0)
    dist_df["distance_01"] = dist_df.groupby("metric")["distance"] \
        .transform(lambda x: (x - x.min()) / (x.max() - x.min()) if (x.max() - x.min()) else 0.0)
    return dist_df


# %% ---------------------------------------------------------------------------
# 5)  Run pipeline over (whole / paired / non‑paired) subsets
# ---------------------------------------------------------------------------
subsets = {
    "whole": df,
    "paired": df[df["paired_status"] == "partner"],
    "non_paired": df[df["paired_status"] == "non-partner"],
}

for tag, subdf in subsets.items():
    print(f"\n=== {tag.upper()}  ({len(subdf)} calls) ===")

    # -- distance table ------------------------------------------------------
    dist_tbl = build_distance_table_spectral(subdf)
    dist_csv = OUT_BASENAME.name.format(tag)
    dist_tbl.to_csv(OUTPUT_DIR / dist_csv, index=False)
    print("saved", dist_csv, "|", len(dist_tbl), "comparisons")

# %% ---------------------------------------------------------------------------
print("\u2705  Pipeline finished. Output files are in:", OUTPUT_DIR.resolve())




=== WHOLE  (3904 calls) ===
  → DTW will be computed for 182 session pairs


saved glm_data_spec_whole.csv | 546 comparisons

=== PAIRED  (1013 calls) ===
  → DTW will be computed for 76 session pairs


saved glm_data_spec_paired.csv | 228 comparisons

=== NON_PAIRED  (2891 calls) ===
  → DTW will be computed for 143 session pairs


saved glm_data_spec_non_paired.csv | 429 comparisons
✅  Pipeline finished. Output files are in: C:\Users\nakul\Desktop\marmoset project\2025_Work_Re-start_Marmoset\2025\glm data
